# Debugging the Currency Conversion Error

This notebook helps diagnose the issue with the currency filter where a `ConversionSyntax` error occurs when converting values to Decimal.

In [ ]:
# Import the necessary libraries
from decimal import Decimal, InvalidOperation, ConversionSyntax
from django.template.defaultfilters import floatformat
import sys
print(f"Python version: {sys.version}")

## Testing the Current Implementation

Let's simulate the function that's causing problems and test different input values.

In [ ]:
def test_currency(value, symbol='R'):
    """
    Test function to simulate our template filter
    """
    print(f"Testing value: '{value}' of type {type(value)}")
    
    if value is None:
        print("Value is None, returning empty string")
        return ''
    
    try:
        value = Decimal(value)
        result = f"{symbol}{floatformat(value, 2)}"
        print(f"Success! Result: {result}")
        return result
    except Exception as e:
        print(f"Error: {type(e).__name__}: {str(e)}")
        return ''

In [ ]:
# Test with various types of values
test_values = [
    100,                  # int
    100.50,               # float
    "100.50",             # string
    Decimal("100.50"),    # Decimal
    "",                   # empty string
    None,                 # None
    "invalid",            # invalid string
    {},                   # dict
    object(),             # custom object
]

for value in test_values:
    print("\n" + "-"*50)
    test_currency(value)

## Implementing a Robust Solution

Based on the tests above, let's create an improved version of the currency filter that handles all edge cases properly.

In [ ]:
def improved_currency(value, symbol='R'):
    """
    Improved currency formatter that's more robust against errors
    """
    if value is None or value == '':
        return f"{symbol}0.00"
    
    try:
        # Handle values that are already decimal or float
        if isinstance(value, (Decimal, float, int)):
            formatted_value = floatformat(value, 2)
            return f"{symbol}{formatted_value}"
        
        # Try to convert string values to Decimal
        if isinstance(value, str):
            value = value.strip()
        
        value = Decimal(value)
        return f"{symbol}{floatformat(value, 2)}"
    except (ValueError, TypeError, InvalidOperation, ConversionSyntax) as e:
        print(f"Handling error: {type(e).__name__}: {str(e)}")
        # Return a safe value for error cases to avoid breaking the display
        return f"{symbol}0.00"

In [ ]:
# Test our improved implementation with the same test values
for value in test_values:
    print("\n" + "-"*50)
    print(f"Testing improved filter with: {value}")
    result = improved_currency(value)
    print(f"Result: {result}")

## Testing with Problematic UserProfile Values

Let's simulate a UserProfile object with different types of `available_balance` values to see how our filter handles them.

In [ ]:
class MockUserProfile:
    def __init__(self, balance):
        self.available_balance = balance

test_profiles = [
    MockUserProfile(100.50),
    MockUserProfile(Decimal('100.50')),
    MockUserProfile("100.50"),
    MockUserProfile(None),
    MockUserProfile(""),
    MockUserProfile({}),
    MockUserProfile(object())
]

for profile in test_profiles:
    print("\n" + "-"*50)
    print(f"Profile balance: {profile.available_balance} of type {type(profile.available_balance)}")
    result = improved_currency(profile.available_balance)
    print(f"Formatted balance: {result}")

## Conclusion

The improved currency filter is more robust and handles a wider range of input types, including errors
that might occur when accessing attributes like `available_balance`. This should fix the issue in the template.